**Cross-Talk Analysis (at NERSC)**
===
### **Author: Hung-Wei,Liu**

First, we import the necessary modules:

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from lgdo import lh5
from dbetto import Props, TextDB
print("done")

done


Next, we set up the path names and file names (for NERSC). We still follow the SOP by extracting the keys from the valid file although we acknowledge the fact that some files are missing:

In [2]:
period, run = "p08", "r015"
xtc_dir = "/global/cfs/cdirs/m2676/data/lngs/l200/scratch/crosstalk_data/xtc"
lmeta = TextDB(path=f"{xtc_dir}/inputs")

valid_file = f"{xtc_dir}/generated/par/valid_keys/l200-{period}-{run}-valid_xtc.json"
valid_keys = list(Props.read_from(valid_file)["valid_keys"])
time_string = valid_keys[0].split("-")[-1]

dsp_dir = f"{xtc_dir}/generated/tier/dsp/xtc/{period}/{run}"
hit_dir = f"{xtc_dir}/generated/tier/hit/xtc/{period}/{run}"
dsp_list = [f"{dsp_dir}/{key}-tier_dsp.lh5" for key in valid_keys]
hit_list = [f"{hit_dir}/{key}-tier_hit.lh5" for key in valid_keys]

The following cell is for removing missing files from the list. It should be removed when the analysis is moved to LNGS.

In [3]:
listed_files = set(os.path.basename(f) for f in hit_list)
actual_files = set(os.listdir(hit_dir))
non_existent_files = listed_files - actual_files
new_hit_list = []
for f in hit_list:
    if os.path.basename(f) not in non_existent_files:
        new_hit_list.append(f)

listed_files = set(os.path.basename(f) for f in dsp_list)
actual_files = set(os.listdir(dsp_dir))
non_existent_files = listed_files - actual_files
new_dsp_list = []
for f in dsp_list:
    if os.path.basename(f) not in non_existent_files:
        new_dsp_list.append(f)

**Extract detector raw ids through TextDB utilities**
---

Not all the groups in the hit_files or dsp_files represent actual detectors. We need to extract the raw ids of detectors in order to choose the correct groups to read out later on.

In [10]:
chmap = lmeta.hardware.configuration.channelmaps.on(time_string)
print(time_string)
geds = [ch for ch in chmap.keys() if chmap[ch]['system']=='geds']
chn_id = []
for detector in geds:
    chn_id.append(chmap[detector]['daq']['rawid'])
print(chn_id)
print(len(chn_id))

20240108T164446Z
[1104000, 1104001, 1104002, 1104003, 1104004, 1104005, 1105600, 1105602, 1105603, 1107202, 1107203, 1107204, 1107205, 1108800, 1108801, 1108802, 1108803, 1108804, 1110402, 1110403, 1110404, 1110405, 1112000, 1112001, 1112002, 1112003, 1112004, 1112005, 1113600, 1113601, 1113602, 1113603, 1113604, 1113605, 1115200, 1115201, 1115202, 1115203, 1115204, 1116801, 1116802, 1116803, 1116804, 1116805, 1118402, 1118403, 1118404, 1118405, 1120000, 1120001, 1120002, 1120003, 1120004, 1120005, 1121600, 1121601, 1121602, 1121603, 1121604, 1121605, 1078400, 1078405, 1080000, 1080001, 1080002, 1080003, 1080004, 1080005, 1081600, 1081601, 1081602, 1081603, 1081604, 1081605, 1083200, 1083201, 1083202, 1083203, 1083204, 1083205, 1084800, 1084801, 1084802, 1084803, 1084804, 1084805, 1086400, 1086401, 1086403, 1086404, 1086405, 1088000, 1088001, 1088002, 1088003, 1088004, 1089600, 1089601, 1089602, 1089603, 1089604]
101


**Step 1: Evaluate Baseline Energy**
---
For each detector, we should extract the events tagged with ```is_baseline == True``` and evaluate the average baseline energy.

In [6]:
baseline_energy = []

baseline_iter_limit = 10
for j, detector in enumerate(chn_id):
    if j > baseline_iter_limit:
        break
    baseline_energy.append(np.mean(relevant_events(
        table_path=f"ch{detector}/hit/",
        files=new_hit_list,
        ene_dataset="cuspEmax_ctc_cal",
        flag_datasets=["is_baseline"],
        conditions={"is_baseline":63})))
    print(f"Baseline energy evaluated for the {j}th detector.")

print(baseline_energy)

Baseline energy evaluated for the 0th detector.
Baseline energy evaluated for the 1th detector.
Baseline energy evaluated for the 2th detector.
Baseline energy evaluated for the 3th detector.
Baseline energy evaluated for the 4th detector.
Baseline energy evaluated for the 5th detector.
Baseline energy evaluated for the 6th detector.
Baseline energy evaluated for the 7th detector.
Baseline energy evaluated for the 8th detector.
Baseline energy evaluated for the 9th detector.
Baseline energy evaluated for the 10th detector.
[np.float64(-0.8561009502910173), np.float64(0.1260089599534006), np.float64(-0.17597473657732793), np.float64(0.12999481028573184), np.float64(-0.10079429025586627), np.float64(-0.14909360718160014), np.float64(0.0730498724317248), np.float64(-0.6125733739494743), np.float64(0.9545673601685722), np.float64(-0.2622581141110762), np.float64(0.9913381211596026)]


**Step 2: Evaluate the cross talk matrix**
---
First, let's define the cross-talk matrix element:

```baseline``` has been evaluated in the previous cell. For E_trig and E_response, we first collect the events that satisfies the selection criteria:

In [8]:
matrix_iter_limit = 2

for j1, trig_detector in enumerate(chn_id):
    if j1 > matrix_iter_limit: 
        break
    energy_1, idxs = relevant_events(
        table_path=f"ch{trig_detector}/hit/",
        files = new_hit_list,
        ene_dataset="cuspEmax_ctc_cal",
        flag_datasets=["is_discharge","is_valid_0vbb_old"],
        conditions={"is_discharge":False,"is_valid_0vbb_old":True},
        energy_range=(1500,4500),
        return_index=True
    )
    print(f"Event selection using #{j1} detector as trigger detector completed.")
    trapTmax_1 = lh5.read(f"ch{trig_detector}/dsp/trapTmax",new_dsp_list,idx=idxs).nda
    print("Trigger energy extraction complete.")
    
    for j2 , resp_detector in enumerate(chn_id):
        if j1 == j2:
            xtalk_matrix[j1,j2]=0
            print("Self interaction ignored. Skipping to next response detector.")
            continue
            
        if j2 > matrix_iter_limit:
            break
        
        energy_2 = lh5.read(f"ch{resp_detector}/hit/cuspEmax_ctc_cal", new_hit_list, idx=idxs).nda
        secondary_selection = (energy_2<100)
        secondary_idxs = idxs[secondary_selection]
        selected_trapTmax_1=trapTmax_1[secondary_selection]
        print("Secondary selection complete. ")
        table_2 = lh5.read(f"ch{resp_detector}/dsp/", new_dsp_list, field_mask=["trapTmin","trapTmax"], idx=secondary_idxs)
        trapTmin_2 = table_2["trapTmin"].nda
        trapTmax_2 = table_2["trapTmax"].nda
        xtalk_matrix[j1,j2] = np.mean(xtalk_element(selected_trapTmax_1,trapTmin_2,baseline_energy[j2]))
        print(f"matrix element {(j1,j2)} evaluation complete")

print(xtalk_matrix[:(matrix_iter_limit+1),:(matrix_iter_limit+1)])

Event selection using #0 detector as trigger detector completed.
Trigger energy extraction complete.
Self interaction ignored. Skipping to next response detector.
Secondary selection complete. 
matrix element (0, 1) evaluation complete
Secondary selection complete. 
matrix element (0, 2) evaluation complete
Event selection using #1 detector as trigger detector completed.
Trigger energy extraction complete.
Secondary selection complete. 
matrix element (1, 0) evaluation complete
Self interaction ignored. Skipping to next response detector.
Secondary selection complete. 
matrix element (1, 2) evaluation complete
Event selection using #2 detector as trigger detector completed.
Trigger energy extraction complete.
Secondary selection complete. 
matrix element (2, 0) evaluation complete
Secondary selection complete. 
matrix element (2, 1) evaluation complete
Self interaction ignored. Skipping to next response detector.
[[ 0.         -0.05266973 -0.04685733]
 [-0.04410721  0.         -0.04803